# Lab 5: Hybrid Search — Best of Both Worlds**Estimated Time:** 10 minutes---

## Step 1: The Limits of Pure Vector Search

In [ ]:
print("=== VECTOR SEARCH: 'Substation Gamma maintenance history' ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 55) AS doc_title,
           ROUND(VECTOR_DISTANCE(c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'Substation Gamma maintenance history'),
               COSINE), 4) AS distance
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY distance
    FETCH APPROXIMATE FIRST 5 ROWS ONLY
""")

## Step 2: Create an Oracle Text Index

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("""
        CREATE INDEX knowledge_chunks_text_idx
        ON city_knowledge_chunks (chunk_text)
        INDEXTYPE IS CTXSYS.CONTEXT
    """)

print("Oracle Text index created successfully.")

In [ ]:
print("=== KEYWORD SEARCH: 'Substation Gamma' ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 55) AS doc_title,
           SCORE(1) AS keyword_score
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    WHERE CONTAINS(c.chunk_text, 'Substation Gamma', 1) > 0
    ORDER BY SCORE(1) DESC
    FETCH FIRST 5 ROWS ONLY
""")

## Step 3: Combine Into Hybrid Search

In [ ]:
print("=== HYBRID SEARCH: 'Substation Gamma maintenance history' ===\n")

QUERY = 'Substation Gamma maintenance history'
KEYWORD_WEIGHT = 0.4
VECTOR_WEIGHT = 0.6

run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 50) AS doc_title,
           ROUND(SCORE(1), 2) AS keyword_score,
           ROUND(1 - VECTOR_DISTANCE(c.embedding,
               VECTOR_EMBEDDING(doc_model USING :query),
               COSINE), 4) AS vector_score,
           ROUND(
               :kw_weight * (SCORE(1) / 100) +
               :vec_weight * (1 - VECTOR_DISTANCE(c.embedding,
                   VECTOR_EMBEDDING(doc_model USING :query),
                   COSINE)),
               4
           ) AS hybrid_score
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    WHERE CONTAINS(c.chunk_text, 'Substation OR Gamma OR maintenance', 1) > 0
       OR VECTOR_DISTANCE(c.embedding,
           VECTOR_EMBEDDING(doc_model USING :query),
           COSINE) < 0.6
    ORDER BY hybrid_score DESC
    FETCH FIRST 5 ROWS ONLY
""", {
    "query": QUERY,
    "kw_weight": KEYWORD_WEIGHT,
    "vec_weight": VECTOR_WEIGHT
})

In [ ]:
def hybrid_search(question, keyword_terms, top_k=5,
                  keyword_weight=0.4, vector_weight=0.6):
    """Perform hybrid keyword + vector search."""
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT c.chunk_id,
                   c.chunk_text,
                   kb.title,
                   ROUND(
                       :kw_weight * (SCORE(1) / 100) +
                       :vec_weight * (1 - VECTOR_DISTANCE(c.embedding,
                           VECTOR_EMBEDDING(doc_model USING :question),
                           COSINE)),
                       4
                   ) AS hybrid_score
            FROM city_knowledge_chunks c
            JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
            WHERE CONTAINS(c.chunk_text, :keywords, 1) > 0
               OR VECTOR_DISTANCE(c.embedding,
                   VECTOR_EMBEDDING(doc_model USING :question),
                   COSINE) < 0.6
            ORDER BY hybrid_score DESC
            FETCH FIRST :top_k ROWS ONLY
        """, {
            "question": question,
            "keywords": keyword_terms,
            "kw_weight": keyword_weight,
            "vec_weight": vector_weight,
            "top_k": top_k
        })

        results = []
        for row in cursor.fetchall():
            chunk_text = row[1].read() if hasattr(row[1], 'read') else row[1]
            results.append({
                "chunk_id": row[0], "text": chunk_text,
                "source": row[2], "score": row[3]
            })
        return results

# Test it
results = hybrid_search(
    "What is the current status of Harbor Bridge?",
    "Harbor OR Bridge OR inspection OR vibration"
)
print(f"Hybrid search returned {len(results)} results:")
for r in results:
    print(f"  [{r['score']}] {r['source'][:60]}")

In [ ]:
question = "Harbor Bridge vibration sensor readings"
keywords = "Harbor OR Bridge OR vibration OR sensor"

print(f"Query: '{question}'\n")

for kw, vec in [(0.2, 0.8), (0.5, 0.5), (0.8, 0.2)]:
    results = hybrid_search(question, keywords,
                            keyword_weight=kw, vector_weight=vec, top_k=3)
    print(f"Keyword={kw:.1f}, Vector={vec:.1f}:")
    for r in results:
        print(f"  [{r['score']}] {r['source'][:55]}")
    print()

Hybrid search gives you the best of both worlds. **Proceed to Lab 6** to use it in a RAG pipeline.